In [1]:
import xarray as xr
import dask
import numpy as np
import os
import time
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=1,name='make_profiles',walltime='00:20:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1','--ntasks-per-node=24'],memory='120GB',interface='ib0') 
print(cluster.job_script()) 

cluster.scale(240) 



#!/usr/bin/env bash

#SBATCH -J make_profiles
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=112G
#SBATCH -t 00:20:00
#SBATCH --constraint=HSW24
#SBATCH --exclusive
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=24
JOB_ID=${SLURM_JOB_ID%;*}



/scratch/cnt0024/hmg2840/albert7a/anaconda3/bin/python -m distributed.cli.dask_worker tcp://172.30.100.1:40139 --nthreads 0 --nprocs 28 --memory-limit 4.29GB --name make_profiles--${JOB_ID}-- --death-timeout 60 --local-directory $SCRATCHDIR/daskjobqueue --interface ib0



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
cluster

In [3]:
import zarr

In [4]:
from dask.distributed import Client
client = Client(cluster)


In [5]:
client

Client Scheduler: tcp://172.30.100.1:40139 Dashboard: http://172.30.100.1:39777/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)


In [ ]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'
gridfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6_lev1.nc4'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})



import Box2x2 as bb


def compute_buoy(t,s):
    rau0  = 1000
    grav  = 9.81
    buoy= -1*(grav/rau0)*sigma0(t,s)
    return buoy

def sigma0(t,s):
    zrau0=1000
    zsr=np.sqrt(np.abs(s))
    zs=s
    zt=t
    zr1 = ( ( ( ( 6.536332e-9*zt-1.120083e-6 )*zt+1.001685e-4)*zt - 9.095290e-3 )*zt+6.793952e-2 )*zt+999.842594
    zr2= ( ( ( 5.3875e-9*zt-8.2467e-7 )*zt+7.6438e-5 ) *zt - 4.0899e-3 ) *zt+0.824493
    zr3= ( -1.6546e-6*zt+1.0227e-4 ) *zt-5.72466e-3
    zr4= 4.8314e-4
    sigma0=( zr4*zs + zr3*zsr + zr2 ) *zs + zr1 - zrau0
    return sigma0

def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=20, dim=['x', 'y'], n=[30, 30])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS




def compute_mean_wprime_bprime(date):
    print('read the data')
    tfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridT_'+date+'-'+date+'.nc'))
    tfile=tfilename[0]
    dst=xr.open_dataset(tfile,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
    tdata=dst['votemper']
    sfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridS_'+date+'-'+date+'.nc'))
    sfile=sfilename[0]
    dss=xr.open_dataset(sfile,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
    sdata=dss['vosaline']
    wfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridW_'+date+'-'+date+'.nc'))
    wfile=wfilename[0]
    dsw=xr.open_dataset(wfile,chunks={'x':1000,'y':1000,'time_counter':1,'depthw':1})
    wdata=dsw['vovecrtz']
    wdata_t=wdata.rename({'depthw':'deptht'})
    print('compute buoyancy')
    buoy=compute_buoy(tdata,sdata)
    print('filter w')
    wprime=filt(wdata_t)
    print('filter buoyancy')
    bprime=filt(buoy)
    wprimebprime=wprime*bprime
    for ibox in bb.boxes:
        box = ibox
        print(box.name)  
        profile_name='/scratch/cnt0024/hmg2840/albert7a/eNATL60/scale/eNATL60'+box.name+'-BLBT02_y'+date[0:4]+'m'+date[4:6]+'d'+date[6:9]+'_wprimebprime-profile.nc'
        #if not os.path.exists(profile_name):
        print('averaging')
        profile=wprimebprime[:,:,box.jmin:box.jmax,box.imin:box.imax].mean(dim={'x','y','time_counter'})
        print('to dataset')
        dataset=profile.to_dataset(name='wprimebprime')
        dataset['wprimebprime'].attrs=tdata.attrs
        dataset['wprimebprime'].attrs['standard_name']='vertical flux of small scales buoyancy' 
        dataset['wprimebprime'].attrs['long_name']='wprimebprime'
        dataset['wprimebprime'].attrs['units']='m2/s3'        
        dataset.attrs['global_attribute']= 'vertical flux of small scales buoyancy profile averaged over 24h and in '+box.name+' computed on occigen with dask-jobqueue '+str(today)
        print('to netcdf')
        dataset.to_netcdf(path=profile_name,mode='w')




%time compute_mean_wprime_bprime('20090701')






Box_1
Box_2
Box_3
Box_4
Box_5
Box_6
Box_7
Box_8
Box_9
Box_10
Box_11
Box_12
Box_13
Box_14
read the data
compute buoyancy
filter w
filter buoyancy
Box_1
averaging
to dataset
to netcdf
Box_2
averaging
to dataset
to netcdf
Box_3
averaging
to dataset
to netcdf
Box_4
averaging
to dataset
to netcdf
Box_5
averaging
to dataset
to netcdf


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:41180 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:41196 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:41254 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:41222 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:41228 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:41234 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:47942 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1

Box_6
averaging
to dataset
to netcdf
Box_7
averaging
to dataset
to netcdf


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:48044 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:48048 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:48050 remote=tcp://172.30.100.1:40139>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:48084 remote=tcp://172.30.100.1:40139>
distributed.utils_perf - WARNING - full garbage collections took 28% CPU time recently (threshold: 10%)


Box_8
averaging
to dataset
to netcdf
Box_9
averaging
to dataset
to netcdf
Box_10
averaging
to dataset
to netcdf


distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://172.30.100.1:54754 remote=tcp://172.30.100.1:40139>
distributed.utils_perf - WARNING - full garbage collections took 25% CPU time recently (threshold: 10%)
